> Try colab version [here](https://colab.research.google.com/drive/1Baon_b4QAE3kL-RpQDbgaFdu9T8rkInH#scrollTo=G2Y24ZE8s0lM).

In [ ]:
# Utils
import os
from time import time
import numpy as np
from itertools import count
from sklearn.metrics import auc as auc_metric


def RetrievalEvaluation(distM, model_label, sketch_label):
    '''
    ## INPUT:
    distM: distance matrix
    model_label: class labels for 3D model
    sketch_label: class labels for 2D sketches

    ##OUTPUT:
    metrics: NN, FT, ST, DCG, E, mAP, pre, rec, auc
    record for visualization (ignored): rankArray
    '''
    distM=distM
    C_sketch = np.zeros((len(sketch_label)), dtype="int32")
    for i in range(len(sketch_label)):
        C_sketch[i] = np.sum(model_label == sketch_label[i])
    C = C_sketch
    recall = np.zeros((distM.shape[0], distM.shape[1]))
    precision = np.zeros((distM.shape[0], distM.shape[1]))
    rankArray = np.zeros((distM.shape[0], distM.shape[1]))

    nb_of_query = C.shape[0]
    p_points = np.zeros((nb_of_query, np.amax(C)))
    ap = np.zeros(nb_of_query)
    nn = np.zeros(nb_of_query)
    ft = np.zeros(nb_of_query)
    st = np.zeros(nb_of_query)
    dcg = np.zeros(nb_of_query)
    e_measure = np.zeros(nb_of_query)

    for qqq in range(nb_of_query):
        temp_dist = np.array(distM[qqq])
        temp_dist = np.squeeze(temp_dist)
        s = list(temp_dist)
        R = sorted(range(len(s)), key=lambda k: s[k])

        model_label_l = model_label[R]
        numRetrieval = distM.shape[1]
        G = np.zeros(numRetrieval)
        rankArray[qqq] = R

        model_label_l = np.squeeze(model_label_l)
        for i in range(numRetrieval):
            if model_label_l[i] == sketch_label[qqq]:
                G[i] = 1
        G_sum = np.cumsum(G)
        r1 = G_sum / float(C[qqq])
        p1 = G_sum / np.arange(1, numRetrieval+1)
        r_points = np.zeros(C[qqq])
        for i in range(C[qqq]):
            temp = np.where(G_sum == i+1)
            r_points[i] = np.where(G_sum == (i+1))[0][0] + 1
        r_points_int = np.array(r_points, dtype=int)

        p_points[qqq][:int(C[qqq])] = G_sum[r_points_int-1] / r_points
        ap[qqq] = np.mean(p_points[qqq][:int(C[qqq])])
        nn[qqq] = G[0]
        ft[qqq] = G_sum[C[qqq]-1] / C[qqq]
        st[qqq] = G_sum[min(2*C[qqq]-1, G_sum.size-1)] / C[qqq]
        p_32 = G_sum[min(31, G_sum.size-1)] / min(32, G_sum.size)
        r_32 = G_sum[min(31, G_sum.size-1)] / C[qqq]
        if p_32 == 0 and r_32 == 0:
            e_measure[qqq] = 0
        else:
            e_measure[qqq] = 2* p_32 * r_32/(p_32+r_32)

        NORM_VALUE = 1 + np.sum(1/np.log2(np.arange(2,C[qqq]+1)))
        dcg_i = 1/np.log2(np.arange(2, len(R)+1)) * G[1:]
        dcg_i = np.insert(dcg_i, 0, G[0])
        dcg[qqq] = np.sum(dcg_i, axis=0)/NORM_VALUE
        recall[qqq] = r1
        precision[qqq] = p1


    nn_av = np.mean(nn)
    ft_av = np.mean(ft)
    st_av = np.mean(st)
    dcg_av = np.mean(dcg)
    e_av = np.mean(e_measure)
    map_ = np.mean(ap)

    pre = np.mean(precision, axis=0)
    rec = np.mean(recall, axis=0)
    auc = auc_metric(rec, pre)

    return nn_av, ft_av, st_av, dcg_av, e_av, map_, p_points, pre, rec, auc, rankArray

## Evaluation

***Put your results here***

In [ ]:
distM_filenames = [
    './sketch_3d_eval/evaluation/team_4_HIT/submission/Task 1/task1.npy',
]

In [ ]:
# Task1: CAD, Task2: Wild

for distM_filename in distM_filenames:
    task_type = distM_filename.split('submission/Task')[-1].split('/')[0].split('v')[0]
    
    # Set the type of task here, or specify your gt_type manually.
    if '1' in task_type:
        gt_type = 'CAD'
    else:
        gt_type = 'WILD'
        
    # gt_type = ['CAD', 'WILD'][0]

    save_dir = distM_filename.split('/submission')[0].replace('evaluation/', 'evaluation_check/')
    distM_file = '{}'.format(distM_filename)
    distM_filename = distM_filename.split('submission/')[-1].replace('/', '--')[:-4]
    try:
        distM = np.load(distM_file)
    except:
        distM = np.loadtxt(distM_file)

    # Put the ground truth files here.
    # Download the GT files from our google-drive: https://drive.google.com/drive/folders/1PFj8DaXB9QV1C8glQpacIMUsjznqASFn
    lbl_model = np.loadtxt('./{}_gt.npy'.format(gt_type))
    lbl_sketch = np.loadtxt('./{}_sketch_gt.npy'.format(gt_type))
    print('distM.shape:', distM.shape)
    print('lbl_model.shape:', lbl_model.shape)
    print('lbl_sketch.shape:', lbl_sketch.shape)


    time_st = time()

    nn_av, ft_av, st_av, dcg_av, e_av, map_, p_points, pre, rec, auc, rankArray = RetrievalEvaluation(distM, lbl_model, lbl_sketch)

    info_1 = 'distM_file: {}\ntime consuming: {}s.\n'.format(distM_file, time() - time_st)
    info_2 = 'nn_av: {}\nft_av: {}\nst_av: {}\ndcg_av: {}\ne_av: {}\nmap_: {}\n'.format(nn_av, ft_av, st_av, dcg_av, e_av, map_)
    info_3 = 'auc: {}\n'.format(auc)
    print(info_1)
    print(info_2)
    print('p_points:\n', p_points)
    print('pre:\n', pre)
    print('rec:\n', rec)
    print(info_3)
    print('rankArray:\n', rankArray)
    print()
    if not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)
    with open(os.path.join(save_dir, '{}_output.txt').format(distM_filename), 'w') as f:
        f.write(info_1 + info_2 + info_3 + '\n')
    np.savetxt(os.path.join(save_dir, '{}_pre.txt'.format(distM_filename)), pre)
    np.savetxt(os.path.join(save_dir, '{}_rec.txt'.format(distM_filename)), rec)
    if gt_type == 'WILD':
        np.save(os.path.join(save_dir, '{}_p_points.npy'.format(distM_filename)), p_points)
        np.save(os.path.join(save_dir, '{}_rankArray.npy'.format(distM_filename)), rankArray)
    else:
        np.save(os.path.join(save_dir, '{}_p_points.npy'.format(distM_filename)), p_points)
        np.save(os.path.join(save_dir, '{}_rankArray.npy'.format(distM_filename)), rankArray)

distM.shape: (23992, 12000)
lbl_model.shape: (12000,)
lbl_sketch.shape: (23992,)
distM_file: /content/drive/MyDrive/MaoJuXiWu/sketch_3d_eval/evaluation/team_4_HIT/submission/Task 1/task1.npy
time consuming: 351.3757801055908s.

nn_av: 0.010795265088362788
ft_av: 0.015414028092297699
st_av: 0.03098713202612801
dcg_av: 0.3628591862434631
e_av: 0.0010532518669274613
map_: 0.020477612465387533

p_points:
 [[1.34770889e-03 1.60000000e-03 2.04081633e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.00000000e-01 6.66666667e-01 5.00000000e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.11111111e-02 7.40740741e-03 8.02139037e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [5.40540541e-04 8.05801773e-04 6.82438581e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.02938503e-04 5.43478261e-04 5.71210967e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.87401575e-04 9.57854406e-04 1.20385233e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+